## Lab 13: Path Planning and Execution (real)

### Plan



**Goal:** Navigate through a series of waypoints as quickly and accurately as possible.

**Process:**
1. Use the Bayes filter to establish a belief about the robot's initial position
2. Find the next waypoint in the list of waypoints to navigate
3. Calculate the control (change in angle, change in distance) needed to get to the waypoint
4. Use turn_degrees() to turn
5. Move forward with ToF-based PID (and ToF averaging because of all the noise)
6. Spin around and localize again to make sure you're on track
7. Repeat until you've seen all the waypoints

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

START_PLOTTER()

2022-05-05 16:25:35,799 | INFO     |: Logger demo_notebook.log initialized.


### Set up the robot 

In [ ]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

        # Data collection stuff
        self.dist_data = []
        self.angle_data = []
        
        # Current pose tracking
        self.current_pose = (0,0,0)

        self.setup_notify()
        

    def setup_notify(self):
        # print("Notifications started")
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        # print(ble.uuid['RX_TOF1'])
        ble.start_notify(ble.uuid['RX_IMU_PITCH'], self.imu_pitch_callback_handler)

    def tof1_callback_handler(self, uuid, byte_array):
        new_tof = self.ble.bytearray_to_float(byte_array)
        # print(f"Got TOF data: {new_tof}")
        self.dist_data.append(new_tof)

    def imu_pitch_callback_handler(self, uuid, byte_array):
        new_angle = self.ble.bytearray_to_float(byte_array)
        # print(f"Got angle data: {new_angle}")
        self.angle_data.append( new_angle )
        
    def set_params(self, pid_p_rot, pid_min_power_rot, pid_p_fwd, pid_min_power_fwd, imu_hz):
        self.ble.send_command(CMD.SET_PID, f"{pid_p_rot}|{pid_min_power_rot}|{pid_p_fwd}|{pid_min_power_fwd}|{imu_hz}"); 

    def set_motor_calib(self, calib_rot, calib_fwd):
        self.ble.send_command(CMD.SET_MOTOR_CALIB, f"{calib_rot}|{calib_fwd}")
        
    def move_distance(self, dist_mm):
        self.ble.send_command(CMD.MOVE_DISTANCE, dist_mm)

    def turn_degrees(self, angle):
        self.ble.send_command(CMD.TURN_DEGREES, angle)

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        return self.current_pose

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # Clear old data if it exists
        self.dist_data = []
        self.angle_data = []
        self.ble.send_command(CMD.OBSERVE, None)
        while len(self.dist_data) < 18 or len(self.angle_data) < 18:
            # print("Data still sending, waiting")
            self.dist_data = []
            self.angle_data = []
            time.sleep(1)
            self.ble.send_command(CMD.GET_DATA, None) # try again for command not sent?
        self.dist_data = [x / 1000 for x in self.dist_data]
        temp_angles = []
        temp_sum = 0
        for i,angle in enumerate(self.angle_data):
            temp_sum -= angle
            temp_angles.append(temp_sum)
        self.angle_data = temp_angles
        print("Observation loop complete")
        print(f"Distance data: {self.dist_data}")
        print(f"Angle data: {self.angle_data}")
        print("")
        return np.array(self.dist_data).reshape((18,1)), np.array(self.angle_data).reshape((18,1))

def ft2meters(pt):
    meters_in_one_foot = 0.3048
    return (pt[0] * meters_in_one_foot, pt[1] * meters_in_one_foot)


def compute_control(cur_pose, prev_pose):
    """ FROM LAB 11:
    Given the current and previous odometry poses, this function extracts
    the control information based on the odometry motion model.

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose 

    Returns:
        [delta_rot_1]: Rotation 1  (degrees)
        [delta_trans]: Translation (meters)
        [delta_rot_2]: Rotation 2  (degrees)
    """
    prev_pose = (prev_pose[0], prev_pose[1], prev_pose[2])
    cur_pose = (cur_pose[0], cur_pose[1], cur_pose[2])
    
    x1,y1,theta1 = prev_pose
    x2,y2,theta2 = cur_pose
    
    delta_rot_1 = np.degrees(np.arctan2(y2-y1, x2-x1))
    delta_trans = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    delta_rot_2 = theta2 - theta1 - delta_rot_1
    
    return mapper.normalize_angle(delta_rot_1), delta_trans, mapper.normalize_angle(delta_rot_2)



### Connect via Bluetooth

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

time.sleep(3)

# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)
print("Ready")


In [ ]:
"""
Test stuff
"""
print("beginning test")
# Params: PID_P_ROT, PID_MIN_ROT, PID_P_FWD, PID_MIN_FWD, IMU_HZ
robot.set_params(1.35, 80, 0.4, 35, 60)
robot.set_motor_calib(1.4, 1.7)
# robot.move_distance(150)
# robot.turn_degrees(90)
print("Test done")

### Run the Bayes Filter for inital localization

In [17]:
# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

# Reset Robot and Plots
# robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
# loc.get_observation_data()

# Run Update Step
# loc.update_step()
# loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom = robot.get_pose()
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-05 16:29:39,233 | INFO     |:  | Number of observations per grid cell: 18
2022-05-05 16:29:39,234 | INFO     |:  | Precaching Views...
2022-05-05 16:29:43,005 | INFO     |:  | Precaching Time: 3.771 secs
2022-05-05 16:29:43,006 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-05 16:29:43,008 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-05-05 16:29:43,210 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-05 16:29:43,210 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


### Follow the trajectory

In [20]:
# points = [
#     (-4, -3),
#     (-2, -1),
#     (1, -1),
#     (2, -3),
#     (5, -3),
#     (5, -2),
#     (5, 3),
#     (0, 3),
#     (0, 0)
# ]

points = [
    (0,0), (0,1), (1,1), (1,0), (0,0)
]

# Assume you perfectly go EVERYWHERE, so you can pre-compute all controls
waypoints = [ft2meters(pt) for pt in points]

for i in range(len(waypoints)-1):
    (start_x, start_y) = waypoints[i]
    (end_x, end_y) = waypoints[i+1]
    (theta1, d, theta2) = compute_control((start_x,start_y,0), (end_x,end_y,0))
    print(f"Control: {(theta1, d, theta2)}")
    
    robot.turn_degrees(theta1)
    robot.move_distance(1000*d) # mm to m
    robot.turn_degrees(theta2)

    # Re-localize for now
    # loc.get_observation_data()
    # loc.update_step()
    # loc.print_update_stats(plot_data=True)
    # current_odom = robot.get_pose()
  

Control: (-90.0, 0.3048, 90.0)


Exception: Not connected to a BLE device